## INICIALIZO EL CLUSTER PARA EJECUTAR SPARK

In [2]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AME Started!
Welcome to AME Version 3.3.9 (January 2022)

SparkSession available as spark and AME available as api

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## IMPORT LAS LIBRERIAS

In [3]:
from pyspark.sql.functions import count , col , lit, desc, when, exp , greatest , least , max , min , round
from pyspark.sql.functions import to_json, struct, collect_list , round, greatest, least, substring, coalesce , udf, monotonically_increasing_id, date_add, datediff, to_date, lit, when, floor, regexp_replace, from_unixtime, unix_timestamp, add_months, col, first, last, sum, min, max, avg, count, round, months_between, countDistinct, asc, desc, row_number, year, month, trim, concat, length, exp , log
from pyspark.sql.types import FloatType, IntegerType, StringType, LongType, DateType, ArrayType , DecimalType
spark.conf.set( "spark.sql.crossJoin.enabled" , "true" )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## FUNCION GENERICA

In [4]:
def archive_date(table, date, partition_name):
    archive = spark.read.format('bigquery').option('table', table).load().filter("ARCHIVE <= '{fecha} 00:00:00'".format(fecha = date)).select(partition_name).distinct().sort(desc(partition_name)).take(1)
    return archive[0].ARCHIVE

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LINEAL

In [5]:
lineal_score_entel = {
"cant_consultas":+0.001926,
"subrango":-0.002686,
"numero_trabajadores":-0.0002429,
"cant_socios":+0.03367,
"ind_bed_socios":+1.056,
"ind_bolcom_socios":+1.008,
"cant_vehic":+0.001573,
"avaluo_uf":-0.0000007575,
"region_ind_rm": -0.1645,
"cantidad_infraccion":-0.00007399,
"deuda_uf_infraccion":-0.00008137,
"cantidad_multa":-0.004963,
"deuda_uf_multa":-0.0003945,
"deuda_telco_uf":+0.00002282,
"score_empresa_telefnc_tile10":-0.1415,
"beta0":+0.5197,
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TRANSFORMACIONES

In [6]:
def transformaciones_score_movil_persona(df_scoren):
    
    df_scoren = (df_scoren.withColumn('dist_clean2' , when((col('kpi_porc_clean2_hh') == 0) | (col('kpi_porc_clean2_hh').isNull()), lit(-0.359))
                                                      .when(col('kpi_porc_clean2_hh') <= 0.5, lit(-0.093))
                                                      .when(col('kpi_porc_clean2_hh') < 1, lit(0.174))
                                                      .when(col('kpi_porc_clean2_hh') == 1, lit(0.538))
                          )
                          .withColumn('avaluo_vehi_hh_t' , when((col('kpi_total_avaluo_vehi_hh_principal') == 0) | (col('kpi_total_avaluo_vehi_hh_principal').isNull()), lit(-0.353))
                                                          .when(col('kpi_total_avaluo_vehi_hh_principal') <= 2185000 , lit(-0.089))
                                                          .when(col('kpi_total_avaluo_vehi_hh_principal') <= 5610000, lit(0.302))
                                                          .when(col('kpi_total_avaluo_vehi_hh_principal') > 5610000, lit(0.691))
                          )
                          .withColumn('renta_percapita' , when((col('kpi_renta_percapita_calc_hh') <= 156378.44) | (col('kpi_renta_percapita_calc_hh').isNull()), lit(-0.546))
                                                          .when(col('kpi_renta_percapita_calc_hh') <= 250120.84 , lit(-0.169))
                                                          .when(col('kpi_renta_percapita_calc_hh') <= 382071.7 , lit(0.201))
                                                          .when(col('kpi_renta_percapita_calc_hh') > 382071.7 , lit(0.607))
                          )
                          .withColumn('cant_tel_cel' , when((col('kpi_cantidad_tel_cel') == 0) | (col('kpi_cantidad_tel_cel').isNull()) , lit(-0.417))
                                                      .when(col('kpi_cantidad_tel_cel') == 1 , lit(-0.139))
                                                      .when(col('kpi_cantidad_tel_cel') == 2 , lit(0.081))
                                                      .when(col('kpi_cantidad_tel_cel') > 2 , lit(0.314))
                          )
                          .withColumn('cant_hijos_menores' , when((col('kpi_cant_hijos_menores') <= 0) | (col('kpi_cant_hijos_menores').isNull()), lit(0.135))
                                                            .when(col('kpi_cant_hijos_menores') <= 2 , lit(0.004))
                                                            .when(col('kpi_cant_hijos_menores') > 2 , lit(-0.332))
                          )
                          .withColumn('padres_abc12' , when((col('kpi_padres_abc12') == 0) | (col('kpi_padres_abc12').isNull()) , lit(0.135))
                                                      .when(col('kpi_padres_abc12') == 1, lit(0.140))
                                                      .when(col('kpi_padres_abc12') == 2, lit(0.595))
                          )
                          .withColumn('ise' , when(col('kpi_ise_new').isin(['AB','C1a','C1b','C2']) , lit(0.779))
                                              .when(col('kpi_ise_new') =='C3' , lit(0.059))
                                              .when(col('kpi_ise_new') =='D' , lit(-0.125))
                                              .when((col('kpi_ise_new').isin(['E','Z','',' ','0'])) | (col('kpi_ise_new').isNull()) , lit(-0.359))
                          )
                          .withColumn('edad_calc' , when((col('dic_edad_26') <= 28) | (col('dic_edad_26').isNull()), lit(-0.294))
                                                    .when(col('dic_edad_26') <= 38 , lit(-0.039))
                                                    .when(col('dic_edad_26') <= 48 , lit(0.113))
                                                    .when(col('dic_edad_26') <= 58 , lit(0.247))
                                                    .when(col('dic_edad_26') > 58 , lit(0.409))
                          )
                          .withColumn('cant_tel_fij' , when((col('dic_cnt_tel_26') <= 0) | (col('dic_cnt_tel_26').isNull()) , lit(-0.186))
                                                    .when(col('dic_cnt_tel_26') > 0 , lit(0.305))
                          )
                          
                )

    return df_scoren


def calculo_score_movil_persona(df_scoren):
     df_scoren = (df_scoren.withColumn('lineal_score_entel' , 0.21666318 + 
                                                  0.792946723652026*col('dist_clean2') + 
                                                  0.610285975629372*col('avaluo_vehi_hh_t') + 
                                                  0.457187772918526*col('renta_percapita') + 
                                                  0.71815752486043*col('cant_tel_cel') + 
                                                  0.808363809320489*col('cant_hijos_menores') + 
                                                  0.447741821812766*col('padres_abc12') + 
                                                  0.225325155749984*col('ise') + 
                                                  0.343630594086875*col('edad_calc') + 
                                                  0.202384574110176*col('cant_tel_fij')
                                    )
                  )
     df_scoren = (df_scoren.withColumn('score_real', (1000 / (1+exp(-1*col('lineal_score_entel')))).astype(IntegerType()))
                            .withColumn('score_movil', floor((1000*(col('score_real')- 188.0021285037991)/716.6010913426493)/2) * 2 + 1)
                            .withColumn('score_movil', when (col('score_movil') < 1 , 1).otherwise(col('score_movil')))
                            .withColumn('score_movil', when (col('score_movil') > 999 , 999).otherwise(col('score_movil')))
                            .withColumn('score_telefonica_movil', when(col('score_movil') == 1 , lit(3))
                                                                  .when(col('score_movil') == 2 , lit(3))
                                                                  .when(col('score_movil') == 30 , lit(29))
                                                                  .when(col('score_movil') == 45 , lit(44))
                                                                  .when(col('dic_tienefdef_26') == 'SI' , lit(0))
                                                                  .when(col('kpi_ind_interdicto') == 'S' , lit(1))
                                                                  .when((col('kpi_fec_nac').isNotNull()) & (~col('kpi_fec_nac').isin(['','19000101'])) & (col('dic_edad_26') < 18), lit(2))
                                                                  .when(((col('kpi_fec_nac').isNull()) | (col('kpi_fec_nac').isin(['','19000101']))), lit(45))
                                                                  #.when(col('dic_rutneg_00') == 1 , lit(30))
                                                                  .otherwise(col('score_movil'))
                            )
                            .withColumn('score_real', col('score_real').astype(IntegerType()))
                            .withColumn('score_real_movil', col('score_movil').astype(IntegerType()))
                            .withColumn('score_ajustado_movil', col('score_telefonica_movil').astype(IntegerType()))
                )

     return df_scoren

    
def transformaciones_score_fija_persona(df_scoren):
    
    df_scoren = (df_scoren.withColumn('dist_clean2' , when((col('kpi_porc_clean2_hh') == 0) | (col('kpi_porc_clean2_hh').isNull()), lit(-0.543))
                                                      .when(col('kpi_porc_clean2_hh') <= 0.45, lit(-0.176))
                                                      .when(col('kpi_porc_clean2_hh') <= 1, lit(0.344))
                          )
                          .withColumn('avaluo_vehi_hh_prom' , when((col('kpi_prom_avaluo_vehi_hh_principal') <= 1571000) | (col('kpi_prom_avaluo_vehi_hh_principal').isNull()), lit(-0.342))
                                                              .when(col('kpi_prom_avaluo_vehi_hh_principal') <= 2587083.38 , lit(0.037))
                                                              .when(col('kpi_prom_avaluo_vehi_hh_principal') <= 4525833.25, lit(0.308))
                                                              .when(col('kpi_prom_avaluo_vehi_hh_principal') <= 5787750 , lit(0.475))
                                                              .when(col('kpi_prom_avaluo_vehi_hh_principal') <= 8007333.25, lit(0.666))
                                                              .when(col('kpi_prom_avaluo_vehi_hh_principal') > 8007333.25, lit(0.880))
                          )
                          .withColumn('renta_hh' , when((col('kpi_renta_total_hh_principal') <= 543257) | (col('kpi_renta_total_hh_principal').isNull()), lit(-0.378))
                                                    .when(col('kpi_renta_total_hh_principal') <= 1028842 , lit(-0.133))
                                                    .when(col('kpi_renta_total_hh_principal') > 1028842 , lit(0.409))
                          )
                          .withColumn('cant_fam_hh' , when(col('kpi_cant_familiares_hh_principal') <= 1 , lit(0.409))
                                                    .when((col('kpi_cant_familiares_hh_principal') > 1) | (col('kpi_cant_familiares_hh_principal').isNull()) , lit(-0.087))
                          )
                          .withColumn('padres_abc12' , when((col('kpi_padres_abc12') == 0) | (col('kpi_padres_abc12').isNull()), lit(-0.189))
                                                    .when(col('kpi_padres_abc12') == 1 , lit(0.240))
                                                    .when(col('kpi_padres_abc12') == 2 , lit(0.906))
                          )
                          .withColumn('cant_hh_lider' , when(col('kpi_cant_hh_es_lider') <= 1 , lit(0.078))
                                                    .when((col('kpi_cant_hh_es_lider') > 1) | (col('kpi_cant_hh_es_lider').isNull()) , lit(-0.448))
                          )
                          .withColumn('renta_efx' , when((col('kpi_renta_efx') <= 153071) | (col('kpi_renta_efx').isNull()), lit(-0.580))
                                                    .when(col('kpi_renta_efx') <= 370646 , lit(-0.187))
                                                    .when(col('kpi_renta_efx') <= 517469 , lit(0.118))
                                                    .when(col('kpi_renta_efx') > 517469 , lit(0.523))
                          )
                          .withColumn('cons_telco_3m' , when((col('kpi_tot_cons_telco_3m') <= 2) | (col('kpi_tot_cons_telco_3m').isNull()), lit(0.138))
                                                    .when((col('kpi_tot_cons_telco_3m') > 2) , lit(-0.198))
                          )
                          .withColumn('ise' , when(col('kpi_ise_new').isin(['AB','C1a','C1b','C2']) , lit(0.576))
                                              .when((col('kpi_ise_new').isin(['C3','D','E','Z',' ','0','','SIN CLASIFICACION'])) | (col('kpi_ise_new').isNull()), lit(-0.139))
                          )
                          .withColumn('edad_calc' , when((col('dic_edad_26') <= 25) | (col('dic_edad_26').isNull()), lit(-0.393))
                                                    .when(col('dic_edad_26') > 25 , lit(0.102))
                          )
                )

    return df_scoren


def calculo_score_fija_persona(df_scoren):
     df_scoren = (df_scoren.withColumn('lineal_score_entel' , 0.52325218 + 
                                                  0.770338868232295*col('dist_clean2') + 
                                                  0.542020389481385*col('avaluo_vehi_hh_prom') + 
                                                  0.785848515826977*col('renta_hh') + 
                                                  1.21154726758955*col('cant_fam_hh') + 
                                                  0.501491480810172*col('padres_abc12') + 
                                                  0.72348428271276*col('cant_hh_lider') +
                                                  0.315931980929232*col('renta_efx') + 
                                                  0.797623166011454*col('cons_telco_3m') + 
                                                  0.371798216113674*col('ise') + 
                                                  0.242318193011601*col('edad_calc')
                                    )
                  )
     df_scoren = (df_scoren.withColumn('score_real', (1000 / (1+exp(-1*col('lineal_score_entel')))).astype(IntegerType()))
                            .withColumn('score_fija', floor(1000*(col('score_real')- 200.20725243995838)/(2*755.9988715971363)) * 2 + 1)
                            .withColumn('score_fija', when (col('score_fija') < 1 , 1).otherwise(col('score_fija')))
                            .withColumn('score_fija', when (col('score_fija') > 999 , 999).otherwise(col('score_fija')))
                            .withColumn('score_ajustado_fijo', when(((col('dic_montototaluf_12') > 1) | (col('dic_montototaluf_02') > 1) | (col('dic_montototaluf_39') > 2)) & (~col('kpi_ise_new').isin(['AB','C1a','C1b','C2','C3'])), lit(51))
                                                            #.when((col('score_fija') < 650) & (col('kpi_ise_new') == 'D') , lit(299))
                                                            .when((col('score_fija') >= 550) & (col('kpi_ise_new') == 'E') , lit(53))
                                                            .when((col('score_fija') >= 500) & (col('score_fija') < 550) & (col('kpi_ise_new') == 'E') , lit(54))
                                                            .when((col('score_fija') < 500) & (col('kpi_ise_new') == 'E') , lit(7))
                                                            .when((col('score_fija') > 50) & ((col('kpi_ise_new') == 'Z') | (col('kpi_ise_new').isNull()) | (col('kpi_ise_new') == '') | (col('kpi_ise_new') == 'SIN CLASIFICACION')) , lit(53))
                                                            .otherwise(col('score_fija'))
                                        )
                            .withColumn('score_fija', col('score_fija').astype(IntegerType()))
                            .withColumn('score_ajustado_fijo', col('score_ajustado_fijo').astype(IntegerType()))
                            .withColumn('score_real', col('score_real').astype(IntegerType()))
                            .withColumn('score_ajustado_fijo', when(col('score_ajustado_fijo') == 1 , lit(3))
                                                            .when(col('score_ajustado_fijo') == 2 , lit(3))
                                                            .when(col('score_ajustado_fijo') == 30 , lit(29))
                                                            .when(col('score_ajustado_fijo') == 45 , lit(44))
                                                            .otherwise(col('score_ajustado_fijo'))
                                        )
                            .withColumn('score_ajustado_fijo', when(col('dic_tienefdef_26') == 'SI', lit(0))
                                                            .when(col('kpi_ind_interdicto') == 'S' , lit(1))
                                                            .when((col('dic_edad_26').isNotNull()) & (col('dic_edad_26') < 18) & (col('ind_pj') == 0), lit(2))
                                                            .when(((col('kpi_fec_nac').isNull()) | (col('kpi_fec_nac').isin(['','19000101']))), lit(45))
                                                            .otherwise(col('score_ajustado_fijo'))
                                        )
                            .withColumn('limite_credito', when(col('score_ajustado_fijo') < 50, lit(0))
                                                            .when(col('score_ajustado_fijo') < 300, lit(25000))
                                                            .when(col('score_ajustado_fijo') < 400, lit(37000))
                                                            .when(col('score_ajustado_fijo') < 550, lit(45000))
                                                            .when(col('score_ajustado_fijo') < 650, lit(63000))
                                                            .otherwise(lit(75000))
                                        )
                  )

     return df_scoren


def valores_por_defecto(df_scoren):
	
  df_scoren = (df_scoren.withColumn('dic_cnt_tel_26', when(col('dic_cnt_tel_26').isNull(), lit('0')).otherwise(col('dic_cnt_tel_26')))
						.withColumn('dic_edad_26', when(col('dic_edad_26').isNull(), lit(0)).otherwise(col('dic_edad_26')))
						.withColumn('dic_est_civil_26', when(col('dic_est_civil_26').isNull(), lit('')).otherwise(col('dic_est_civil_26')))
						.withColumn('dic_tienefdef_26', when(col('dic_tienefdef_26').isNull(), lit('')).otherwise(col('dic_tienefdef_26')))
						.withColumn('dic_totavaluo_32', when(col('dic_totavaluo_32').isNull(), lit(0)).otherwise(col('dic_totavaluo_32')))
						.withColumn('dic_rutneg_00', when(col('dic_rutneg_00').isNull(), lit(1)).otherwise(col('dic_rutneg_00')))
						.withColumn('dic_nroacreed_39', when(col('dic_nroacreed_39').isNull(), lit(0)).otherwise(col('dic_nroacreed_39')))
						.withColumn('dic_fechnac2_26', when(col('dic_fechnac2_26').isNull(), lit('00000000')).otherwise(col('dic_fechnac2_26')))
						.withColumn('dic_nombre_00', when(col('dic_nombre_00').isNull(), lit('')).otherwise(col('dic_nombre_00')))
						.withColumn('dic_codigoactividad_26', when(col('dic_codigoactividad_26').isNull(), lit(0)).otherwise(col('dic_codigoactividad_26')))
						.withColumn('dic_region_18', when(col('dic_region_18').isNull(), lit('')).otherwise(col('dic_region_18')))
						.withColumn('dic_tot_clqveh_10', when(col('dic_tot_clqveh_10').isNull(), lit(0)).otherwise(col('dic_tot_clqveh_10')))
						.withColumn('dic_cod_profesion_26', when(col('dic_cod_profesion_26').isNull(), lit(0)).otherwise(col('dic_cod_profesion_26')))
						.withColumn('dic_rutctacte_06', when(col('dic_rutctacte_06').isNull(), lit(0)).otherwise(col('dic_rutctacte_06')))
						.withColumn('dic_tipodomicilio_18', when(col('dic_tipodomicilio_18').isNull(), lit('N/A')).otherwise(col('dic_tipodomicilio_18')))
						.withColumn('dic_sexo_26', when(col('dic_sexo_26').isNull(), lit('')).otherwise(col('dic_sexo_26')))
						.withColumn('dic_ind_ae_23', when(col('dic_ind_ae_23').isNull(), lit('0')).otherwise(col('dic_ind_ae_23')))
						.withColumn('dic_vehiculo_furgon', when(col('dic_vehiculo_furgon').isNull(), lit(0)).otherwise(col('dic_vehiculo_furgon')))
						.withColumn('dic_tot_vehs_10', when(col('dic_tot_vehs_10').isNull(), lit(0)).otherwise(col('dic_tot_vehs_10')))
						.withColumn('dic_dirciudad_18', when(col('dic_dirciudad_18').isNull(), lit(0)).otherwise(col('dic_dirciudad_18')))
						.withColumn('dic_camionetas_10', when(col('dic_camionetas_10').isNull(), lit(0)).otherwise(col('dic_camionetas_10')))
						.withColumn('dic_predictorpub_06', when(col('dic_predictorpub_06').isNull(), lit(0)).otherwise(col('dic_predictorpub_06')))
						.withColumn('dic_actividad_26', when(col('dic_actividad_26').isNull(), lit('PROFESION NO CODIFICADA')).otherwise(col('dic_actividad_26')))
						.withColumn('dic_finiconrut_38', when(col('dic_finiconrut_38').isNull(), lit('00000000')).otherwise(col('dic_finiconrut_38')))
						.withColumn('dic_fechvenc_02', when(col('dic_fechvenc_02').isNull(), lit('00000000')).otherwise(col('dic_fechvenc_02')))
						.withColumn('dic_fec_const_23', when(col('dic_fec_const_23').isNull(), lit('00000000')).otherwise(col('dic_fec_const_23')))
						.withColumn('dic_totcons3_38', when(col('dic_totcons3_38').isNull(), lit(0)).otherwise(col('dic_totcons3_38')))
						.withColumn('dic_nrootros_02', when(col('dic_nrootros_02').isNull(), lit(0)).otherwise(col('dic_nrootros_02')))
						.withColumn('dic_autos_sw_10', when(col('dic_autos_sw_10').isNull(), lit(0)).otherwise(col('dic_autos_sw_10')))
						.withColumn('vehi_avaluo_total_hh', when(col('vehi_avaluo_total_hh').isNull(), lit(0)).otherwise(col('vehi_avaluo_total_hh')))
						.withColumn('renta_total_hh', when(col('renta_total_hh').isNull(), lit(0)).otherwise(col('renta_total_hh')))
						.withColumn('avaluo_bbrr_total_hh', when(col('avaluo_bbrr_total_hh').isNull(), lit(0)).otherwise(col('avaluo_bbrr_total_hh')))
						.withColumn('clean2_hh', when(col('clean2_hh').isNull(), lit(1)).otherwise(col('clean2_hh')))
						.withColumn('predictor1_max_hh', when(col('predictor1_max_hh').isNull(), lit(0)).otherwise(col('predictor1_max_hh')))
						.withColumn('kpi_ise_efx', when(col('kpi_ise_efx').isNull(), lit('')).otherwise(col('kpi_ise_efx')))
						.withColumn('kpi_ind_interdicto', when(col('kpi_ind_interdicto').isNull(), lit('N')).otherwise(col('kpi_ind_interdicto')))
						.withColumn('kpi_renta_efx', when(col('kpi_renta_efx').isNull(), lit(0)).otherwise(col('kpi_renta_efx')))
						.withColumn('kpi_geo_clean2', when(col('kpi_geo_clean2').isNull(), lit('1')).otherwise(col('kpi_geo_clean2')))
						.withColumn('kpi_ind_bank', when(col('kpi_ind_bank').isNull(), lit(0)).otherwise(col('kpi_ind_bank')))
						.withColumn('kpi_cant_hijos_menores', when(col('kpi_cant_hijos_menores').isNull(), lit(0)).otherwise(col('kpi_cant_hijos_menores')))
						.withColumn('kpi_vehi_avaluo', when(col('kpi_vehi_avaluo').isNull(), lit('0')).otherwise(col('kpi_vehi_avaluo')))
						.withColumn('kpi_cantidad_tel_cel', when(col('kpi_cantidad_tel_cel').isNull(), lit(0)).otherwise(col('kpi_cantidad_tel_cel')))
						.withColumn('kpi_cantidad_tel_fijo', when(col('kpi_cantidad_tel_fijo').isNull(), lit(0)).otherwise(col('kpi_cantidad_tel_fijo')))
						.withColumn('kpi_antiguedad_vehi', when(col('kpi_antiguedad_vehi').isNull(), lit('-99')).otherwise(col('kpi_antiguedad_vehi')))
						.withColumn('kpi_redcode8', when(col('kpi_redcode').isNull(), lit('NULL')).otherwise(col('kpi_redcode')))
						.withColumn('kpi_fec_nac', when(col('kpi_fec_nac').isNull(), lit('19000101')).otherwise(col('kpi_fec_nac')))
						.withColumn('kpi_rubro', when(col('kpi_rubro').isNull(), lit('NULL')).otherwise(col('kpi_rubro')))
						.withColumn('kpi_ind_bed_socios', when(col('kpi_ind_bed_socios').isNull(), lit('0')).otherwise(col('kpi_ind_bed_socios')))
						.withColumn('kpi_ind_bolcom_socios', when(col('kpi_ind_bolcom_socios').isNull(), lit('0')).otherwise(col('kpi_ind_bolcom_socios')))
						.withColumn('kpi_empresa_activa', when(col('kpi_empresa_activa').isNull(), lit('0')).otherwise(col('kpi_empresa_activa')))
						.withColumn('kpi_cantidad_socios', when(col('kpi_cantidad_socios').isNull(), lit('0')).otherwise(col('kpi_cantidad_socios')))
						.withColumn('kpi_cantidad_trabajador', when(col('kpi_cantidad_trabajador').isNull(), lit('0')).otherwise(col('kpi_cantidad_trabajador')))
						.withColumn('kpi_subrango', when(col('kpi_subrango').isNull(), lit('NULL')).otherwise(col('kpi_subrango')))
						#.withColumn('score_ajustado_movil', when(col('score_ajustado_movil').isNull(), lit(0)).otherwise(col('score_ajustado_movil')))
						#.withColumn('score_pm', when(col('score_pm').isNull(), lit(0)).otherwise(col('score_pm')))
						#.withColumn('score_sum_pm', when(col('score_sum_pm').isNull(), lit('1.000000')).otherwise(col('score_sum_pm')))
						.withColumn('dic_montototaluf_02', when(col('dic_montototaluf_02').isNull(), lit(0)).otherwise(col('dic_montototaluf_02')))
						.withColumn('dic_montototaluf_12', when(col('dic_montototaluf_12').isNull(), lit(0)).otherwise(col('dic_montototaluf_12')))
						.withColumn('dic_montototaluf_39', when(col('dic_montototaluf_39').isNull(), lit(0)).otherwise(col('dic_montototaluf_39')))
						.withColumn('dic_totdoc_12', when(col('dic_totdoc_12').isNull(), lit(0)).otherwise(col('dic_totdoc_12')))
						.withColumn('dic_totdocs_39', when(col('dic_totdocs_39').isNull(), lit(0)).otherwise(col('dic_totdocs_39')))
						.withColumn('dic_totdocs_02', when(col('dic_totdocs_02').isNull(), lit(0)).otherwise(col('dic_totdocs_02')))
						#.withColumn('score_real_fijo', when(col('score_real_fijo').isNull(), lit(0)).otherwise(col('score_real_fijo')))
						#.withColumn('score_ajustado_fijo', when(col('score_ajustado_fijo').isNull(), lit(0)).otherwise(col('score_ajustado_fijo')))
						#.withColumn('limite_credito', when(col('limite_credito').isNull(), lit(0)).otherwise(col('limite_credito')))
						#.withColumn('p_score1', when(col('p_score1').isNull(), lit(1)).otherwise(col('p_score1')))
						#.withColumn('score_real_movil', when(col('score_real_movil').isNull(), lit(0)).otherwise(col('score_real_movil')))
						#.withColumn('score_pyme', when(col('score_pyme').isNull(), lit(0)).otherwise(col('score_pyme')))
						#.withColumn('p_score1_pyme', when(col('p_score1_pyme').isNull(), lit(1)).otherwise(col('p_score1_pyme')))
		)
  return df_scoren


def valores_menores_edad(df_scoren):
    df_scoren = (df_scoren.withColumn('dic_cnt_tel_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51) , lit('0')).otherwise(col('dic_cnt_tel_26')))
                          .withColumn('dic_edad_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_edad_26')))
                          .withColumn('dic_est_civil_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('dic_est_civil_26')))
                          #.withColumn('dic_tienefdef_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('dic_tienefdef_26')))
                          .withColumn('dic_totavaluo_32', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_totavaluo_32')))
                          .withColumn('dic_rutneg_00', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(1)).otherwise(col('dic_rutneg_00')))
                          .withColumn('dic_nroacreed_39', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_nroacreed_39')))
                          .withColumn('dic_fechnac2_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('00000000')).otherwise(col('dic_fechnac2_26')))
                          .withColumn('dic_nombre_00', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('dic_nombre_00')))
                          .withColumn('dic_codigoactividad_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_codigoactividad_26')))
                          .withColumn('dic_region_18', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('dic_region_18')))
                          .withColumn('dic_tot_clqveh_10', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_tot_clqveh_10')))
                          .withColumn('dic_cod_profesion_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_cod_profesion_26')))
                          .withColumn('dic_rutctacte_06', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_rutctacte_06')))
                          .withColumn('dic_tipodomicilio_18', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('N/A')).otherwise(col('dic_tipodomicilio_18')))
                          .withColumn('dic_sexo_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('dic_sexo_26')))
                          .withColumn('dic_ind_ae_23', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('dic_ind_ae_23')))
                          .withColumn('dic_vehiculo_furgon', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_vehiculo_furgon')))
                          .withColumn('dic_tot_vehs_10', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_tot_vehs_10')))
                          .withColumn('dic_dirciudad_18', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_dirciudad_18')))
                          .withColumn('dic_camionetas_10', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_camionetas_10')))
                          .withColumn('dic_predictorpub_06', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_predictorpub_06')))
                          .withColumn('dic_actividad_26', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('PROFESION NO CODIFICADA')).otherwise(col('dic_actividad_26')))
                          .withColumn('dic_finiconrut_38', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('00000000')).otherwise(col('dic_finiconrut_38')))
                          .withColumn('dic_fechvenc_02', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('00000000')).otherwise(col('dic_fechvenc_02')))
                          .withColumn('dic_fec_const_23', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('00000000')).otherwise(col('dic_fec_const_23')))
                          .withColumn('dic_totcons3_38', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_totcons3_38')))
                          .withColumn('dic_nrootros_02', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_nrootros_02')))
                          .withColumn('dic_autos_sw_10', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_autos_sw_10')))
                          .withColumn('vehi_avaluo_total_hh', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('vehi_avaluo_total_hh')))
                          .withColumn('renta_total_hh', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('renta_total_hh')))
                          .withColumn('avaluo_bbrr_total_hh', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('avaluo_bbrr_total_hh')))
                          .withColumn('clean2_hh', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(1)).otherwise(col('clean2_hh')))
                          .withColumn('predictor1_max_hh', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('predictor1_max_hh')))
                          .withColumn('kpi_ise_efx', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('')).otherwise(col('kpi_ise_efx')))
                          .withColumn('kpi_ind_interdicto', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('N')).otherwise(col('kpi_ind_interdicto')))
                          .withColumn('kpi_renta_efx', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('kpi_renta_efx')))
                          .withColumn('kpi_geo_clean2', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('1')).otherwise(col('kpi_geo_clean2')))
                          .withColumn('kpi_ind_bank', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('kpi_ind_bank')))
                          .withColumn('kpi_cant_hijos_menores', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('kpi_cant_hijos_menores')))
                          .withColumn('kpi_vehi_avaluo', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_vehi_avaluo')))
                          .withColumn('kpi_cantidad_tel_cel', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('kpi_cantidad_tel_cel')))
                          .withColumn('kpi_cantidad_tel_fijo', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('kpi_cantidad_tel_fijo')))
                          .withColumn('kpi_antiguedad_vehi', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('-99')).otherwise(col('kpi_antiguedad_vehi')))
                          .withColumn('kpi_redcode', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('NULL')).otherwise(col('kpi_redcode')))
                          #.withColumn('kpi_fec_nac', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('19000101')).otherwise(col('kpi_fec_nac')))
                          .withColumn('kpi_rubro', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('NULL')).otherwise(col('kpi_rubro')))
                          .withColumn('kpi_ind_bed_socios', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_ind_bed_socios')))
                          .withColumn('kpi_ind_bolcom_socios', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_ind_bolcom_socios')))
                          .withColumn('kpi_empresa_activa', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_empresa_activa')))
                          .withColumn('kpi_cantidad_socios', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_cantidad_socios')))
                          .withColumn('kpi_cantidad_trabajador', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('0')).otherwise(col('kpi_cantidad_trabajador')))
                          .withColumn('kpi_subrango', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('NULL')).otherwise(col('kpi_subrango')))
                          #.withColumn('score_ajustado_movil', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_ajustado_movil')))
                          #.withColumn('score_pm', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_pm')))
                          #.withColumn('score_sum_pm', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit('1.000000')).otherwise(col('score_sum_pm')))
                          .withColumn('dic_montototaluf_02', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_montototaluf_02')))
                          .withColumn('dic_montototaluf_12', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_montototaluf_12')))
                          .withColumn('dic_montototaluf_39', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_montototaluf_39')))
                          .withColumn('dic_totdoc_12', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_totdoc_12')))
                          .withColumn('dic_totdocs_39', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_totdocs_39')))
                          .withColumn('dic_totdocs_02', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('dic_totdocs_02')))
                          #.withColumn('score_real_fijo', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_real_fijo')))
                          #.withColumn('score_ajustado_fijo', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_ajustado_fijo')))
                          #.withColumn('limite_credito', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('limite_credito')))
                          #.withColumn('p_score1', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(1)).otherwise(col('p_score1')))
                          #.withColumn('score_real_movil', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_real_movil')))
                          #.withColumn('score_pyme', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(0)).otherwise(col('score_pyme')))
                          #.withColumn('p_score1_pyme', when((col('dic_edad_26') < 18) & (col('ind_pj') == 0) & (col('score_ajustado_fijo') != 51), lit(1)).otherwise(col('p_score1_pyme')))
                  )
    df_scoren = (df_scoren.withColumn('clean2_hh', col('clean2_hh').cast(DecimalType(18,2)))
                        .withColumn('dic_montototaluf_02', col('dic_montototaluf_02').cast(DecimalType(18,2)))
                        .withColumn('dic_montototaluf_12', col('dic_montototaluf_12').cast(DecimalType(18,2)))
                        .withColumn('dic_montototaluf_39', col('dic_montototaluf_39').cast(DecimalType(18,2)))
                        .withColumn('dic_fec_const_23', when(col('dic_fec_const_23') == '0', lit('00000000')).otherwise(col('dic_fec_const_23')))
                        .withColumn('dic_edad_26', when(col('ind_defunc') == 1, lit(0)).otherwise(col('dic_edad_26')))
                        .withColumn('dic_est_civil_26', when(col('dic_est_civil_26') == '', lit('NO REGISTRA')).otherwise(col('dic_est_civil_26')))
              )

    return df_scoren

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LLAMAR TABLAS PARA UTILIZAR

In [17]:
#CONSOLIDADO PERSONA (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.monedas_norm_dev_dummy", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.monedas_norm_dev_dummy").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("uf")


archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consolidado_persona")

query_consolidado_persona = """
		        WITH uf_hoy as (
		                SELECT valor_uf
		                FROM(
		                    SELECT valor_uf, row_number() over(ORDER BY fecha desc) as rooooow1 
		                    FROM uf) x
		                WHERE x.rooooow1 = 1
		            ) 
				SELECT A.rutp AS rut,
					   CEILING(datediff(now() ,CASE WHEN fnac_26 = '00000000' THEN NULL ELSE fnac_26 END)/365.25) as dic_edad_26,
					   COALESCE(UPPER(est_civil), '') as dic_est_civil_26,
					   CASE WHEN ind_defunc = 1 THEN 'SI' ELSE 'NO' END as dic_tienefdef_26,
					   valor_uf as DIC_VALORUF_00,
					   nombre as dic_nombre_00,
					   COALESCE(nacr_39, 0) as dic_nroacreed_39,
					   COALESCE(regexp_replace(fnac_26, '-', ''), '00000000') as dic_fechnac2_26,
					   COALESCE(codigo_acti, 0) as dic_codigoactividad_26,
					   '00000000' as dic_fec_const_23,
					   CASE WHEN region = 'DE LOS LAGOS' THEN 'LOS LAGOS'
								WHEN region = 'DE LOS RIOS' THEN 'LOS RIOS'
								WHEN region = 'DE MAGALLANES Y ANTARTICA CHILENA' THEN 'MAGALLANES Y DE LA ANTARTICA CHILENA'
								WHEN region = 'DE NUBLE' THEN 'NUBLE'
								WHEN region = 'DE ATACAMA' THEN 'ATACAMA'
								WHEN region = 'DE ANTOFAGASTA' THEN 'ANTOFAGASTA'
								WHEN region = 'DE COQUIMBO' THEN 'COQUIMBO'
								WHEN region = 'DE LA ARAUCANIA' THEN 'LA ARAUCANIA'
								WHEN region = 'DE TARAPACA' THEN 'TARAPACA'
								WHEN region = 'DE ARICA y PARINACOTA' THEN 'ARICA Y PARINACOTA'
								WHEN region = 'DE VALPARAISO' THEN 'VALPARAISO'
								WHEN region = 'DEL BIO BIO' THEN 'BIO-BIO'
								WHEN region = 'DEL LIBERTADOR BERNARDO OHIGGINS' THEN "LIBERTADOR GENERAL BERNARDO O'HIGGINS"
								WHEN region = 'DEL MAULE' THEN 'MAULE'
								WHEN region = 'AYSEN DEL GENERAL CARLOS IBANEZ' THEN 'AYSEN DEL GRAL. CARLOS IBANEZ DEL CAMPO'
								ELSE coalesce(region, '') END AS dic_region_18,
					   COALESCE(cant_vehic, 0) as dic_tot_clqveh_10,
					   cast(COALESCE(codigo_prof, 0) as int) as dic_cod_profesion_26,
					   COALESCE(nccc, 0) as dic_rutctacte_06,
					   CASE WHEN tip_ultdir = '1' THEN 'PARTICULAR'
							WHEN tip_ultdir = '2' THEN 'COMERCIAL'
							WHEN tip_ultdir = '3' THEN 'CASA MATRIZ'
							WHEN tip_ultdir = '4' THEN 'SUCURSAL'
							WHEN tip_ultdir = '5' THEN 'OTRO'
							ELSE 'N/A' END as dic_tipodomicilio_18,
					   COALESCE(sexo, '') AS dic_sexo_26, 
					   COALESCE(vehi_furgon, 0) as dic_vehiculo_furgon,
					   COALESCE(vehi_autos, 0) as dic_vehiculo_aut,
					   COALESCE(tot_vehi_exp, 0) as dic_tot_vehs_10,
					   RTRIM(LTRIM(des_ultcom)) as dic_dircomuna_18,
					   '' as dic_rubro_23,
					   RTRIM(LTRIM(resto_dir)) as dic_direccion_18,
					   RTRIM(LTRIM(des_ultciu)) as dic_dirciudad_18,
					   COALESCE(vehi_camionetas, 0) as dic_camionetas_10,
					   COALESCE(actividad, 'PROFESION NO CODIFICADA')  as dic_actividad_26,
					   COALESCE(regexp_replace(fecha_ven_02, '-', ''), '00000000') as dic_fechvenc_02,
					   COALESCE(dic_totcons3_38, 0) as dic_totcons3_38,
					   COALESCE(otro_02, 0) as dic_nrootros_02,
					   COALESCE(vehi_station_auto, 0) as dic_autos_sw_10,
					   CAST(cast (coalesce(avaluo,0) as float) / cast (valor_uf as float) AS DECIMAL(17,2)) as dic_totavaluo_32,
					   CAST(tmouf_02 AS DECIMAL(17,2)) as dic_montototaluf_02,
					   CAST(tmot_12 AS DECIMAL(17,2)) as dic_montototaluf_12,
					   CAST(tmot_39 AS DECIMAL(17,2)) as dic_montototaluf_39,
					   COALESCE(tdoc_12, 0) as dic_totdoc_12,
					   COALESCE(tdoc_39, 0) as dic_totdocs_39,
					   COALESCE(tdoc_02, 0) as dic_totdocs_02,
					   1 as dic_rutneg_00,
                	   '' as dic_rutneg_glo_00,
                	   consultas_3m as kpi_tot_cons_telco_3m,
                	   0 as ind_pj,
                	   ind_defunc
				FROM (SELECT * FROM consolidado_persona WHERE archive = '2022-05-30') A
				LEFT JOIN uf_hoy B
				ON 1 = 1
"""

df_cons_per = spark.sql(query_consolidado_persona)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#TABLA KPI (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.kpi_prod_ref_telefonica_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.kpi_prod_ref_telefonica_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("kpi")

query_kpi = """
      SELECT 
			rut,
			COALESCE(regexp_replace(KPI_CANT_TEL, '"', ''), '0') AS dic_cnt_tel_26,
			COALESCE(regexp_replace(KPI_NUM_RUBROS_PN, '"', ''), '0') AS dic_ind_ae_23,
			COALESCE(regexp_replace(kpi_ise_ignite, '"', ''), '') AS kpi_ise_new,
			COALESCE(regexp_replace(kpi_ise_ignite, '"', ''), '') AS kpi_ise_efx,
			COALESCE(regexp_replace(KPI_IND_INTERDICTO, '"', ''), 'N') AS kpi_ind_interdicto,
			CAST(COALESCE(regexp_replace(KPI_RENTA_EFX, '"', ''), '0') AS BIGINT) AS kpi_renta_efx,
			COALESCE(regexp_replace(KPI_GEO_CLEAN2, '"', ''), '1') AS kpi_geo_clean2,
			CAST(COALESCE(regexp_replace(KPI_IND_BANK, '"', ''), '0') AS BIGINT) AS kpi_ind_bank,
			CAST(COALESCE(regexp_replace(KPI_CANT_HIJOS_MENORES, '"', ''), '0') AS BIGINT) AS kpi_cant_hijos_menores,
			COALESCE(regexp_replace(KPI_VEHI_AVALUO, '"', ''), '0') AS kpi_vehi_avaluo,
			cast(COALESCE(regexp_replace(KPI_CANTIDAD_TEL_CEL, '"', ''), '0') AS BIGINT) AS kpi_cantidad_tel_cel,
			cast(COALESCE(regexp_replace(KPI_CANTIDAD_TEL_FIJO, '"', ''), '0') AS BIGINT) AS kpi_cantidad_tel_fijo,
			COALESCE(regexp_replace(KPI_ANTIGUEDAD_VEHI, '"', ''), '-99') AS kpi_antiguedad_vehi,
			COALESCE(regexp_replace(KPI_REDCODE, '"', ''), 'NULL') AS kpi_redcode,
			COALESCE(regexp_replace(KPI_FEC_NAC, '"', ''), '19000101') AS kpi_fec_nac,
			COALESCE(regexp_replace(KPI_RUBRO, '"', ''), 'NULL') AS kpi_rubro,
			COALESCE(regexp_replace(KPI_IND_BED_SOCIOS, '"', ''), '0') AS kpi_ind_bed_socios,
			COALESCE(regexp_replace(KPI_IND_BOLCOM_SOCIOS, '"', ''), '0') AS kpi_ind_bolcom_socios,
			COALESCE(regexp_replace(KPI_EMPRESA_ACTIVA, '"', ''), '0') AS kpi_empresa_activa,
			COALESCE(regexp_replace(KPI_CANTIDAD_SOCIOS, '"', ''), '0') AS kpi_cantidad_socios,
			COALESCE(regexp_replace(KPI_CANTIDAD_TRABAJADOR, '"', ''), '0') AS kpi_cantidad_trabajador,
			COALESCE(regexp_replace(KPI_SUBRANGO, '"', ''), 'NULL') AS kpi_subrango,
            cast(regexp_replace(KPI_TOTAL_AVALUO_VEHI_HH_PRINCIPAL, '"', '') as bigint) as kpi_total_avaluo_vehi_hh_principal,
            cast(regexp_replace(kpi_clean2_hh, '"', '') as float) as kpi_porc_clean2_hh,
            cast(regexp_replace(kpi_renta_per_capita_calc_hh_principal, '"', '') as bigint) as kpi_renta_percapita_calc_hh,
            0 as kpi_padres_abc12,
            cast(regexp_replace(kpi_renta_total_hh_principal, '"', '') as bigint) as kpi_renta_total_hh_principal,
            cast(regexp_replace(kpi_prom_avaluo_vehi_hh_principal, '"', '') as float) as kpi_prom_avaluo_vehi_hh_principal,
            cast(regexp_replace(kpi_cant_familiares_hh_principal, '"', '') as bigint) as kpi_cant_familiares_hh_principal,
            cast(regexp_replace(kpi_cant_hh_es_lider, '"', '') as bigint) as kpi_cant_hh_es_lider
      FROM kpi
"""

df_kpi = spark.sql(query_kpi)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#CONSULTAS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consultas_telefonica_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consultas_telefonica_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consultas")

query_consultas = """
		SELECT  rut,
		        max(regexp_replace(fecha_consulta, '-', '')) as dic_finiconrut_38
      	FROM consultas
      	GROUP BY rut
"""

df_consultas = spark.sql(query_consultas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#CONSOLIDADO SCORE (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_scores_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_scores_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("score")

query_score = """
		SELECT  rutp as rut,
		        predictor1 as dic_predictorpub_06
      	FROM score
      	WHERE archive = '2022-05-30'
"""

df_score = spark.sql(query_score)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#HOUSEHOLD (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.hh_principal_hist_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.hh_principal_hist_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("household")

query_household = """
		SELECT rut,
		       total_avaluo_vehi_hh_principal as vehi_avaluo_total_hh,
			   renta_total_hh_principal as renta_total_hh,
			   bbrr_avaluo_total_hh_principal as avaluo_bbrr_total_hh,
			   porc_personas_clean2_hh_principal as clean2_hh,
			   predictor1_max_hh_principal as predictor1_max_hh
      	FROM household
      	WHERE archive = '2022-05-30'
"""

df_household = spark.sql(query_household)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#SCORE PYME (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.score_pyme_telefonica_cambrian", '2022-06-03' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.score_pyme_telefonica_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("score_pyme")

query_score_pyme = """
		SELECT 
		       score_pyme as score_pm
      	FROM score_pyme
      	WHERE archive = '2022-06-03'
"""

df_score_pyme = spark.sql(query_score_pyme)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#RUBRO PERSONAS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd", '2022-04-26' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("rubro_personas")

query_rubro_personas = """
			SELECT s.rutp as rut
				,	s.desc_acteco as dic_rubro_23_piv
				from ( select ps.rutp 
					        , ps.activ
					        , dir.cod_acteco
					        , dir.desc_acteco
					        , dir.fecha_inicio_acteco
					        , row_number() over(partition by ps.rutp order by dir.fecha_inicio_acteco desc) as file
					    from rubro_personas ps
					    LATERAL VIEW explode(ps.actividades_economicas) adTable as dir
    					where archive = '2022-04-26'
    				)s
			WHERE s.file = 1
"""

df_rubro_personas = spark.sql(query_rubro_personas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## INFORMACION CONFIGURE

In [14]:
# -- coding: UTF-8 --

# PROD
# 0: debug
# 1: production
# 2: new columns
ind_prod = 0

# Bases genericas
cl_efx_att = 'cl_efx_att'
cl_efx_pre_att = 'cl_efx_pre_att'
cl_efx_prod = 'cl_efx_prod'

# Formato
format0 = 'yyyyMM'
format1 = 'yyyy-MM'
formatoHora = '%H:%M:%S'
formatoFechaEstandar = '%Y%m%d'
formatoFechaEstandar1 = '%Y-%m-%d'
regexFecha = '^([0-9]{4}[-/]?((0[13-9]|1[012])[-/]?(0[1-9]|[12][0-9]|30)|(0[13578]|1[02])[-/]?31|02[-/]?(0[1-9]|1[0-9]|2[0-8]))|([0-9]{2}(([2468][048]|[02468][48])|[13579][26])|([13579][26]|[02468][048]|0[0-9]|1[0-6])00)[-/]?02[-/]?29)$'

# Tabla
tb_monedas_norm = 'monedas_norm'
tb_consolidado_per = 'consolidado'
tb_consolidado_emp = 'consolidado_empresa'
tb_kpi = 'kpi_prod_ref'
tb_consultas = 'consultas_bureau'
tb_predictor = 'consolidado_scores'
tb_hh_principal = 'hh_principal_hist'
tb_score_pyme = 'consolidado_scores_emp'
tb_personas_semanal = 'personas_semanal'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## JOIN

In [18]:
df_1 = (df_cons_per.join(df_kpi, "rut" , how = 'left')
                   .join(df_score, "rut" , how = 'left')
                   .join(df_consultas, "rut" , how = 'left')
                   .join(df_household, "rut" , how = 'left')
                   .join(df_rubro_personas, "rut" , how = 'left')
                   .crossJoin(df_score_pyme)
       )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TRANSFORMACIONES


In [19]:
df1 = transformaciones_score_movil_persona(df_1)
df2 = transformaciones_score_fija_persona(df1)
df3 = calculo_score_movil_persona(df2)
df4 = calculo_score_fija_persona(df3)
df5 = valores_por_defecto(df4)
df = valores_menores_edad(df5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## VER DATA FRAME

In [20]:
df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+----------------+----------------+--------------+--------------------+----------------+---------------+----------------------+----------------+--------------------+-----------------+--------------------+----------------+--------------------+-----------+-------------------+----------------+---------------+----------------+------------+--------------------+----------------+-----------------+----------------+---------------+---------------+---------------+---------------+----------------+-------------------+-------------------+-------------------+-------------+--------------+--------------+-------------+-----------------+---------------------+------+----------+--------------+-------------+-----------+-----------+------------------+-------------+--------------+------------+----------------------+---------------+--------------------+---------------------+-------------------+-----------+-----------+---------+------------------+---------------------+---------

## ESCRIBIR DATOS

In [21]:
df.coalesce(1).write.mode('overwrite').csv('gs://dfa-dna-ws0011-la-prd-sandbox/contingencia_telefonica_semanal/contingencia_telefonica_semanal')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…